In [1]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from ogb.graphproppred import GraphPropPredDataset
from torch_geometric.loader import DataLoader

# Download and process data at './dataset/ogbg_molfreesolv/'
dataset = GraphPropPredDataset(name = 'ogbg-molfreesolv')

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

Downloaded 0.00 GB: 100%|██████████| 2/2 [00:00<00:00, 11.57it/s]


Extracting dataset/freesolv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 642/642 [00:00<00:00, 31816.70it/s]

Saving...


In [2]:
print(dataset[0][0].keys())

dict_keys(['edge_index', 'edge_feat', 'node_feat', 'num_nodes'])


In [9]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0

In [10]:
for g, label in tqdm(dataset):
    node_list.append(np.arange(g["num_nodes"]) + num_nodes)  # All the nodes are considered in a single graph
    edges.append(np.stack(g["edge_index"]).T + num_nodes)
    edge_feats.append(g["edge_feat"])
    node_feats.append(g["node_feat"])
    labels.append(label)
    num_nodes += g["num_nodes"]

100%|██████████| 642/642 [00:00<00:00, 37280.12it/s]


In [11]:
_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [12]:
_node_list = sparse.lil_matrix((len(dataset), num_nodes))
for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

In [13]:
data = {
    "node_feats": node_feats,
    "edge": _edges,
    "graph_class": _labels,
    "edge_feats": _edge_feats,
    "node_list": _node_list,
}
np.savez_compressed("ogbg-molfreesolv.npz", **data)

/Users/tuyiwen/Library/Python/3.8/lib/python/site-packages/numpy/lib/npyio.py:696: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [14]:
task_data={
    "train" :train_idx,
    "val" :valid_idx,
    "test" :test_idx
}
np.savez_compressed("ogbg-molfreesolv_task.npz",**task_data)